In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import seaborn as sns

In [2]:
%cd /home/stepan/kaggle/cars_from_auction

/home/stepan/kaggle/cars_from_auction


In [3]:
%ls

data/  models/  prepare/  result/  review/  test/  train/


In [4]:
%ls data/

processed/  sample_submission.csv  test.csv  train.csv  zipcodes.csv


In [5]:
data_path = 'data/processed/'
data_file = 'filled_transformed_test.csv'

In [6]:
data = pd.read_csv(data_path + data_file, index_col=0)

In [7]:
data

,engine_capacity,type,registration_year,gearbox,power,model,mileage,fuel,brand,damage,zipcode,insurance_price
60314,1.600000,small car,2013,manual,136.0,swift,40000,gasoline,suzuki,0.0,30449,6.194405
12566,3.207037,coupé,2004,auto,333.0,6er,150000,gasoline,bmw,0.0,45307,6.507278
17760,2.105880,station wagon,2006,auto,170.0,e_klasse,150000,diesel,mercedes_benz,0.0,59494,6.131226
8876,1.639746,limousine,1999,manual,101.0,astra,150000,gasoline,opel,0.0,25524,5.438079
80392,1.322234,limousine,1975,manual,54.0,andere,150000,diesel,mercedes_benz,0.0,70794,7.012115
...,...,...,...,...,...,...,...,...,...,...,...,...
93878,1.400000,limousine,1999,manual,86.0,corolla,150000,gasoline,toyota,0.0,44339,4.700480
99783,2.200458,station wagon,2002,auto,184.0,3er,150000,diesel,bmw,0.0,47574,5.480639
57399,1.308723,small car,2005,manual,52.0,fox,100000,gasoline,volkswagen,0.0,50389,4.094345
97106,1.977524,bus,2001,manual,151.0,transporter,150000,diesel,volkswagen,0.0,12209,6.835185


In [8]:
zip_data = pd.read_csv('data/zipcodes.csv', index_col=0)

In [9]:
zip_data

,zipcode,city,latitude,longitude
0,19348,Berge bei Perleberg,53.237460,11.870770
1,85309,Pörnbach,48.616700,11.466700
2,24790,"Osterrönfeld Heidkrug, Gemeinde Osterrönfeld",54.275360,9.737535
3,98646,Hildburghausen,50.439501,10.723922
4,27336,"Frankenfeld, Aller",52.769510,9.430780
...,...,...,...,...
17370,48465,Isterberg,52.363460,7.160770
17371,14929,Treuenbrietzen Bardenitz,52.070188,12.937977
17372,14913,Treuenbrietzen Bardenitz,52.070188,12.937977
17373,22946,Dahmker,53.587820,10.444710


# Merge

In [10]:
def merge_data(data, zipcodes):
    zipcodes = zipcodes.drop_duplicates(keep='last', subset=['zipcode'])
    mdata = data.reset_index().merge(zipcodes, on="zipcode", how="left").set_index('index')
    return mdata

In [11]:
new_data = merge_data(data, zip_data)

In [12]:
new_data

,engine_capacity,type,registration_year,gearbox,power,model,mileage,fuel,brand,damage,zipcode,insurance_price,city,latitude,longitude
index,,,,,,,,,,,,,,,
60314,1.600000,small car,2013,manual,136.0,swift,40000,gasoline,suzuki,0.0,30449,6.194405,Hannover,52.384470,9.726930
12566,3.207037,coupé,2004,auto,333.0,6er,150000,gasoline,bmw,0.0,45307,6.507278,"Essen, Ruhr",51.462488,7.008645
17760,2.105880,station wagon,2006,auto,170.0,e_klasse,150000,diesel,mercedes_benz,0.0,59494,6.131226,"Soest, Westfalen",51.566980,8.110620
8876,1.639746,limousine,1999,manual,101.0,astra,150000,gasoline,opel,0.0,25524,5.438079,Itzehoe,53.926752,9.512700
80392,1.322234,limousine,1975,manual,54.0,andere,150000,diesel,mercedes_benz,0.0,70794,7.012115,Filderstadt,48.666700,9.216670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93878,1.400000,limousine,1999,manual,86.0,corolla,150000,gasoline,toyota,0.0,44339,4.700480,Dortmund,51.504225,7.483654
99783,2.200458,station wagon,2002,auto,184.0,3er,150000,diesel,bmw,0.0,47574,5.480639,Goch,51.679880,6.156480
57399,1.308723,small car,2005,manual,52.0,fox,100000,gasoline,volkswagen,0.0,50389,4.094345,"Wesseling, Rheinland",50.833300,6.983330


In [13]:
new_data.describe()

,engine_capacity,registration_year,power,mileage,damage,zipcode,insurance_price,latitude,longitude
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,49952.000000,49952.000000
mean,1.807091,2003.733600,125.771408,125278.300000,0.077640,51196.608000,5.472985,50.873700,9.436050
std,0.449847,7.299557,58.547123,39509.609835,0.267607,25750.460527,0.962283,1.705111,2.068292
min,0.100000,1945.000000,1.000000,5000.000000,0.000000,1067.000000,2.302585,47.410100,5.916670
25%,1.464101,2000.000000,82.000000,100000.000000,0.000000,30926.000000,4.787492,49.466700,7.738370
50%,1.774235,2004.000000,118.000000,150000.000000,0.000000,50170.000000,5.438079,51.000000,9.037355
75%,2.000000,2008.000000,150.000000,150000.000000,0.000000,71720.750000,6.086775,52.199974,10.883300
max,4.335217,2016.000000,500.000000,150000.000000,1.000000,99998.000000,10.858807,54.937195,14.970560


In [14]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 60314 to 478
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   engine_capacity    50000 non-null  float64
 1   type               50000 non-null  object 
 2   registration_year  50000 non-null  int64  
 3   gearbox            50000 non-null  object 
 4   power              50000 non-null  float64
 5   model              50000 non-null  object 
 6   mileage            50000 non-null  int64  
 7   fuel               50000 non-null  object 
 8   brand              50000 non-null  object 
 9   damage             50000 non-null  float64
 10  zipcode            50000 non-null  int64  
 11  insurance_price    50000 non-null  float64
 12  city               49952 non-null  object 
 13  latitude           49952 non-null  float64
 14  longitude          49952 non-null  float64
dtypes: float64(6), int64(3), object(6)
memory usage: 6.1+ MB


In [15]:
new_data.to_csv(data_path + 'merged_' + data_file)